In [30]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc # Permettez-nous de commander par ordre décroissant
sc=SparkContext(appName="TweetApp6") # Ne peut executer q'une seul fois. redémarrez votre noyau pour détecter les erreurs .
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc) # Permettez-nous d'utiliser SQL pour qeury

In [31]:
ssc.checkpoint( "file:///home/dataera/Bureau/tweet2")
socket_stream = ssc.socketTextStream("127.0.0.1", 5532)
lines = socket_stream.window( 20 )
from collections import namedtuple
fields = ("tag", "count" )

In [32]:
Tweet = namedtuple( 'Tweet', fields ) # Chaque élément se verra attribuer un champ
# Utilisez des parenthèses pour plusieurs lignes ou utilisez \.
( lines.flatMap( lambda text: text.split( " " ) ) #Divisé à une liste
  .filter( lambda word: word.lower().startswith("#") ) # Vérifie les hashtag
  .map( lambda word: ( word.lower(), 1 ) ) # Minuscules le mot, met en place un tuple
  .reduceByKey( lambda a, b: a + b )  # Reduces by key
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Mettre dans un objet Tweet
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Trie par ordre décroissant par nombre
              .limit(10).registerTempTable("tweets") ) ) # Pour chaque dix tweets sera enregistré comme une table.

In [33]:
sqlContext
ssc.start()

In [34]:
import matplotlib.pyplot as plt #Bibliothèque de visualisation
import seaborn as sn #Bibliothèque de visualisation

In [35]:
get_ipython().magic(u'matplotlib inline')

import time 

In [36]:
import csv
def on_status(self, status): 
    with open('file.txt', 'w') as f: 
        f.write('Author,Date,Text')
        writer = csv.writer(f)
        writer.writerow([status.author.screen_name, status.created_at, status.text])

In [40]:

with open("output.csv", "a") as csv_file:
    csv.writer(csv_file).writerow(["ID_STR", "CREATED_AT", "SCREEN_NAME","TEXT", "HASHTAGS"])
    for tweet in Tweet:
        print (tweet)['id_str'],\
        tweet['created_at'],\
        tweet['user']['screen_name'].encode('utf-8'),\
        tweet['text'].encode('utf-8'),\
        [hashtag['text'].encode('utf-8') for hashtag in tweet['entities']\['hashtags'] ]
        
        for hashtag in tweet['entities']['hashtags']:
            csv.writer(csv_file).writerow([
            tweet['id_str'],
            tweet['created_at'],
            tweet['user']['screen_name'].encode('utf-8'),
            tweet['text'].encode('utf-8'),
            hashtag['text'].encode('utf-8')])

SyntaxError: unexpected character after line continuation character (<ipython-input-40-b417c9fcbb49>, line 5)

In [6]:
top_100_tweets = sqlContext.sql( "Select tag, count from tweets" )
print(top_100_tweets.show(100))

+--------------+-----+
|           tag|count|
+--------------+-----+
|#cybersecurity|    9|
|   #daytrading|    7|
|       #market|    7|
|       #stocks|    7|
|       #nasdaq|    7|
|         #nyse|    7|
|      #infosec|    5|
|       #google|    3|
|         #data|    2|
|        #cy…rt|    2|
+--------------+-----+

None


In [ ]:
from IPython import display #Nous permet d'afficher des éléments dans le notebook
count = 0
while count < 100:
    time.sleep( 20 )
    top_100_tweets = sqlContext.sql( "Select tag, count from tweets" )
    top_100_df = top_100_tweets.toPandas() # Bibliothèque de Dataframe
    display.clear_output(wait=True) #Efface la sortie, si un plot existe.
    plt.figure( figsize = ( 10, 8) )
    sn.barplot( x="count", y="tag", data=top_100_df)
    plt.show()
    count = count + 1

In [29]:
ssc.stop()